In [1]:
import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import UniSkip
from config import *
from datetime import datetime, timedelta

from tensorboardX import SummaryWriter

In [2]:
language = "english"
d = DataLoader("./dataset/"+language+"/corpus.txt")

Loading text file at ./dataset/english/corpus.txt
Making dictionary for these words
Using cached dictionary at ./dataset/english/corpus.txt.pkl
Making reverse dictionary


In [3]:
mod = UniSkip()
if USE_CUDA:
    mod.cuda(CUDA_DEVICE)

In [4]:
lr = 3e-4
optimizer = torch.optim.Adam(params=mod.parameters(), lr=lr)

In [5]:
writer = SummaryWriter()

In [8]:
loss_trail = []
last_best_loss = None

def debug(i, loss, prev, nex, prev_pred, next_pred):
    global loss_trail
    global last_best_loss
    global current_time

    this_loss = loss.item()
    loss_trail.append(this_loss)
    loss_trail = loss_trail[-20:]
    
    try:
        trail_loss = sum(loss_trail)/len(loss_trail)
        if last_best_loss is None or last_best_loss > trail_loss:
            print("Loss improved from {} to {}".format(last_best_loss, trail_loss))
            
            save_loc = "./saved_models/skip-best".format(lr, VOCAB_SIZE)
            print("saving model at {}".format(save_loc))
            torch.save(mod.state_dict(), save_loc)
            
            last_best_loss = trail_loss
    except Exception as e:
       print("Couldn't save model because {}".format(e))

In [9]:
print("Starting training...")

# a million iterations
for i in range(0, 1000000):
    sentences, lengths = d.fetch_batch(8 * 8)

    loss, prev, nex, prev_pred, next_pred  = mod(sentences, lengths)
    writer.add_scalar('data/loss', loss, i)
    

    if i % 30 == 0:
        str_prev = d.convert_indices_to_sentences(prev)
        str_prev_pred = d.convert_indices_to_sentences(prev_pred)
        str_next = d.convert_indices_to_sentences(nex)
        str_next_pred = d.convert_indices_to_sentences(next_pred)
        writer.add_text('Prev', str_prev + ' | ' + str_prev_pred, i)
        writer.add_text('Next', str_next + ' | ' + str_next_pred, i)
        debug(i, loss, prev, nex, prev_pred, next_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Starting training...
Loss improved from None to 19.80767059326172
saving model at ./saved_models/skip-best
Couldn't save model because [Errno 2] No such file or directory: './saved_models/skip-best'
Loss improved from None to 18.183279037475586
saving model at ./saved_models/skip-best
Couldn't save model because [Errno 2] No such file or directory: './saved_models/skip-best'
Loss improved from None to 17.534679412841797
saving model at ./saved_models/skip-best
Couldn't save model because [Errno 2] No such file or directory: './saved_models/skip-best'
Loss improved from None to 17.203145027160645
saving model at ./saved_models/skip-best
Couldn't save model because [Errno 2] No such file or directory: './saved_models/skip-best'
Loss improved from None to 16.998593139648438
saving model at ./saved_models/skip-best
Couldn't save model because [Errno 2] No such file or directory: './saved_models/skip-best'
Loss improved from None to 16.860496520996094
saving model at ./saved_models/skip-bes

KeyboardInterrupt: 